In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers
import keras

%matplotlib inline
tf.random.set_seed(42)

2024-02-17 15:21:27.413067: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-17 15:21:27.415875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 15:21:27.454819: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-17 15:21:27.455480: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 15:21:28.299946: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import tensorflow as tf
#import tensorflow_addons as tfa

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
#from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from keras import Sequential
from keras import layers
from keras.models import Model
from keras.layers import LSTM, BatchNormalization, Dropout, Dense, Flatten, Conv1D
from keras.layers import MaxPooling1D, GRU, Input,Masking, Concatenate, dot
from keras.optimizers import Adam, SGD
from keras.losses import MeanAbsoluteError
from keras.metrics import RootMeanSquaredError
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import mean_squared_error

#### Train Dataset 
#### BID_RA_19_20.xlsx

In [3]:
df = pd.read_excel(r"~/Downloads/BID_RA_data/BID_RA_19_20.xlsx")

In [4]:
df.dtypes

Order ID                                  int64
Order Date                       datetime64[ns]
FY_OrderDate                             object
Month_Name_OrderDate                     object
Purchase Mode                            object
Bid start date                   datetime64[ns]
Bid end date                     datetime64[ns]
HOD ID                                   object
Seller ID                                object
Order_Quarter                            object
Order MSME Verified                      object
MSME verfied gender wise                 object
MSME verified social category            object
Startup verfied                          object
Sub Category ID                          object
Expected delivery date           datetime64[ns]
Shipment received date                   object
Order Qty                                 int64
Order value                             float64
Consignment ID                            int64
GeM Invoice Number                      

In [5]:
df1 = df.drop(df.columns[[0, 7, 19, 20, 21, 31]], axis=1)

In [6]:
df1["price"] = df1["Order value"]/df1["Order Qty"]

In [7]:
import datetime
from datetime import datetime as dt

In [8]:
df1["Time taken to deliver"] = (df1["Expected delivery date"] - df1["Order Date"]).dt.days

In [9]:
df1.drop(columns=["Month_Name_OrderDate", "Order_Quarter", "Invoice Date","CRAC date", "Payment Date"], inplace=True)

In [10]:
columns_to_encode = ["Purchase Mode", "MSME verfied gender wise", "Order MSME Verified", "Startup verfied"]

In [11]:
le = LabelEncoder()

In [12]:
for column in columns_to_encode:
       df1[column+'_encoded'] = le.fit_transform(df1[column])

In [13]:
df1 = df1.drop(columns=["Purchase Mode", "MSME verfied gender wise", "Order MSME Verified", "Startup verfied"])

In [14]:
df1 = df1.drop(columns=["FY_OrderDate", "Bid start date", "Bid end date"])

In [15]:
df1["Payment Type"+'_encoded'] = le.fit_transform(df1["Payment Type"])

In [16]:
df1["MSME verified social category"+'_encoded'] = le.fit_transform(df1["MSME verified social category"])

In [17]:
df1 = df1.drop(columns=["MSME verified social category", "Payment Type"])

In [18]:
df1 = df1.drop(columns=["Order Date", "Expected delivery date", "Shipment received date"])

In [19]:
df1 = df1.drop(columns=['Seller ID'])

In [20]:
df1["Sub Category ID"].value_counts()

Sub Category ID
home_info_comp_comp_de52008682          811482
home_comm_nonm_peda_bicy                 89195
home_comm_moto_auto_ambu                 61019
home_appa_foot_sh88838715_sh71256827      8535
home_furn_acco_furn_almi                  7739
                                         ...  
home_mine_fabr_spec_cott                     1
home_medi_medi_me16126272_me82121660         1
home_offi_of64586874_draf_geom               1
home_appa_pers_bath_swac                     1
home_info_data_netw_outd                     1
Name: count, Length: 1352, dtype: int64

In [21]:
df2=df1[df1["Sub Category ID"]=="home_info_comp_comp_de52008682"]

In [22]:
df2 = df2.drop(columns=['Sub Category ID'])

In [23]:
X = df2.drop(columns=['Time taken to deliver'])
y = df2['Time taken to deliver']

In [24]:
X.columns

Index(['Order Qty', 'Order value', 'Shipment Quantity Received',
       'Shipment Quantity Rejected', 'Shipment Quantity Accepted',
       'CRAC amount', 'Total payments', 'price', 'Purchase Mode_encoded',
       'MSME verfied gender wise_encoded', 'Order MSME Verified_encoded',
       'Startup verfied_encoded', 'Payment Type_encoded',
       'MSME verified social category_encoded'],
      dtype='object')

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
X_train.head()

,Order Qty,Order value,Shipment Quantity Received,Shipment Quantity Rejected,Shipment Quantity Accepted,CRAC amount,Total payments,price,Purchase Mode_encoded,MSME verfied gender wise_encoded,Order MSME Verified_encoded,Startup verfied_encoded,Payment Type_encoded,MSME verified social category_encoded
1038441,1432,74870688.0,1,0,1,52284.0,52284.0,52284.0,0,0,1,0,1,0
933364,1432,74870688.0,2,0,2,104568.0,52284.0,52284.0,0,0,1,0,1,0
741645,1432,74870688.0,1,0,1,52284.0,52284.0,52284.0,0,0,1,0,1,0
524745,1432,74870688.0,2,0,2,104568.0,52284.0,52284.0,0,0,1,0,1,0
293495,693,19339551.0,2,0,2,55814.0,50511.0,27907.0,0,0,1,0,3,0


In [27]:
X_train_bil, y_train_bil = np.array(X_train), np.array(y_train)
X_test_bil, y_test_bil = np.array(X_test), np.array(y_test)
#X_train_bil = X_train_bil.reshape((X_train_bil.shape[0], 1, X_train_bil.shape[1]))
#X_test_bil = X_test_bil.reshape((X_test_bil.shape[0], 1, X_test_bil.shape[1]))
print(X_train_bil.shape, y_train_bil.shape, X_test_bil.shape, y_test_bil.shape)

(649185, 14) (649185,) (162297, 14) (162297,)


In [38]:
from keras.callbacks import EarlyStopping

In [29]:
max_features = 10000
max_len = 50

## LSTM-GRU Architecture

In [39]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU
import np_utils
from keras.layers import Dense, LSTM, Bidirectional
from keras.layers import Embedding
from keras.layers import GRU
model_GRU = Sequential()
#model_GRU.add(Bidirectional(LSTM(units=50, dropout=0.5, return_sequences=True)))
model_GRU.add(LSTM(units=50, input_shape=(14,1), return_sequences=True))
model_GRU.add(Dropout(0.3))
model_GRU.add(GRU(units=50, input_shape=(14,1),return_sequences=False))
model_GRU.add(Activation('tanh'))
model_GRU.add(Dropout(0.3))
model_GRU.add(Dense(1))
model_GRU.add(Activation('relu'))
model_GRU.compile(loss='mse', optimizer='adam', metrics = ('MAPE'))

In [40]:
model_GRU.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 14, 50)            10400     
                                                                 
 dropout_3 (Dropout)         (None, 14, 50)            0         
                                                                 
 gru_3 (GRU)                 (None, 50)                15300     
                                                                 
 activation_6 (Activation)   (None, 50)                0         
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
 activation_7 (Activation)   (None, 1)                

In [41]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=60,verbose=1)]

In [42]:
history_GRU = model_GRU.fit(X_train_bil, y_train_bil, validation_data=(X_test_bil, y_test_bil), batch_size = 128, epochs = 60, callbacks=callbacks)

Epoch 1/60
5072/5072 [==============================] - 71s 14ms/step - loss: 54.3635 - MAPE: 56238.4336 - val_loss: 18.9887 - val_MAPE: 6.0756
Epoch 2/60
5072/5072 [==============================] - 68s 13ms/step - loss: 17.3175 - MAPE: 47166.9883 - val_loss: 0.4883 - val_MAPE: 0.3731
Epoch 3/60
5072/5072 [==============================] - 66s 13ms/step - loss: 6.8698 - MAPE: 46365.2852 - val_loss: 0.4744 - val_MAPE: 0.8454
Epoch 4/60
5072/5072 [==============================] - 67s 13ms/step - loss: 5.5750 - MAPE: 46090.5625 - val_loss: 0.4146 - val_MAPE: 0.5069
Epoch 5/60
5072/5072 [==============================] - 67s 13ms/step - loss: 4.4100 - MAPE: 48604.2852 - val_loss: 0.4051 - val_MAPE: 0.5838
Epoch 6/60
5072/5072 [==============================] - 67s 13ms/step - loss: 3.3950 - MAPE: 49413.9648 - val_loss: 0.4174 - val_MAPE: 0.7868
Epoch 7/60
5072/5072 [==============================] - 68s 13ms/step - loss: 2.5399 - MAPE: 42116.7734 - val_loss: 0.3729 - val_MAPE: 0.5130
Epo

## Running LSTM-GRU Hybrid model on test data

In [43]:
pred_GRU = model_GRU.predict(X_test_bil)

5072/5072 [==============================] - 12s 2ms/step


In [44]:
from sklearn.metrics import mean_absolute_error
from keras.layers import Bidirectional
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [46]:
# Create a DataFrame of Real and Predicted values
prices_gru_seen = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": pred_GRU.ravel()
}, index = df1.index[-len(y_test): ]) 
prices_gru_seen.head()

,Real,Predicted
886268,30,30.035341
886269,30,30.009228
886270,30,30.009228
886271,30,30.035341
886272,28,28.032667


## Catboost Implementation on test data

In [47]:
from catboost import CatBoostRegressor
model_cb = CatBoostRegressor(iterations=1000,  # Adjust the number of iterations as needed
                             learning_rate=0.01,  # Adjust the learning rate as needed
                             depth=10,  # Adjust the depth of the trees
                             loss_function='RMSE',  # Specify the loss function for regression
                             random_seed=42,
                            verbose=100)
model_cb.fit(X_train_bil, y_train_bil)
y_pred_cb = model_cb.predict(X_test_bil)
from sklearn.metrics import mean_squared_error
mse_cat = mean_squared_error(y_test_bil, y_pred_cb)
rmse_cat = np.sqrt(mse_cat)
print(f"Root Mean Squared Error: {rmse_cat}")
print(f"Mean Squared Error: {mse_cat}")

0:	learn: 4.3073639	total: 160ms	remaining: 2m 40s
100:	learn: 1.6317269	total: 5.48s	remaining: 48.7s
200:	learn: 0.7042132	total: 10.9s	remaining: 43.1s
300:	learn: 0.4348597	total: 16.3s	remaining: 37.8s
400:	learn: 0.3670875	total: 21.8s	remaining: 32.5s
500:	learn: 0.3420945	total: 27.3s	remaining: 27.2s
600:	learn: 0.3274832	total: 32.9s	remaining: 21.8s
700:	learn: 0.3180081	total: 38.4s	remaining: 16.4s
800:	learn: 0.3087991	total: 43.9s	remaining: 10.9s
900:	learn: 0.2992131	total: 49.5s	remaining: 5.43s
999:	learn: 0.2894523	total: 55.2s	remaining: 0us
Root Mean Squared Error: 0.3828428766051127
Mean Squared Error: 0.14656866816727754


In [48]:
# Create a DataFrame of Real and Predicted values
prices_catb_seen = pd.DataFrame({
    "Real": y_test.ravel(),
    "Predicted": y_pred_cb.ravel()
}, index = df1.index[-len(y_test): ]) 
prices_catb_seen.head()

,Real,Predicted
886268,30,30.001874
886269,30,30.002854
886270,30,30.002557
886271,30,30.001874
886272,28,28.001359


## ON UNSEEN DATA 

In [49]:
df22 = pd.read_excel(r"~/Downloads/BID_RA_data/BID_RA_22_23.xlsx")

In [50]:
df22_1 = df22.drop(df22.columns[[0, 7, 19, 20, 21, 31 ]], axis=1)
df22_1["price"] = df22_1["Order value"]/df22_1["Order Qty"]
df22_1["Time taken to deliver"] = (df22_1["Expected delivery date"] - df22_1["Order Date"]).dt.days
df22_1.drop(columns=["Month_Name_OrderDate", "Order_Quarter", "Invoice Date","CRAC date", "Payment Date"], inplace=True)
for column in columns_to_encode:
       df22_1[column+'_encoded'] = le.fit_transform(df22_1[column])
df22_1 = df22_1.drop(columns=["FY_OrderDate", "Bid start date", "Bid end date"])
df22_1["Payment Type"+'_encoded'] = le.fit_transform(df22_1["Payment Type"])
df22_1["MSME verified social category"+'_encoded'] = le.fit_transform(df22_1["MSME verified social category"])

df22_1= df22_1.drop(columns=['Seller ID'])
df22_1 = df22_1.drop(columns=["MSME verified social category", "Payment Type"])
df22_1 = df22_1.drop(columns=["Order Date", "Expected delivery date", "Shipment received date"])
df22_1 = df22_1.drop(columns=["Purchase Mode", "Order MSME Verified", "MSME verfied gender wise", "Startup verfied"])
df22_1["Sub Category ID"].value_counts()
#df22_1=df22_1[df22_1["Sub Category ID"]=="home_comm_nonm_peda_bicy"]
#print(df22_1.shape)
#df22_1 = df22_1.drop(columns=["Sub Category ID"])
#I = df22_1.drop(columns=['Time taken to deliver'])
#j = df22_1['Time taken to deliver']
#I_train, I_test, j_train, j_test = train_test_split(I, j, test_size=0.2)
#I_train_bil, j_train_bil = np.array(I_train), np.array(j_train)
#I_test_bil, j_test_bil = np.array(I_test), np.array(j_test)

#print(I_train_bil.shape, j_train_bil.shape, I_test_bil.shape, j_test_bil.shape)

Sub Category ID
home_info_comp_comp_de52008682          305420
home_info_comp_co36451542_inte          172531
home_elec_elec_powe_line                170724
home_elec_elec_powe_onli                170494
home_univ_univ_univ_univ                133236
                                         ...  
home_powe_elec_elec_weld                     1
home_dome_cons_audi_di68048835               1
home_dist_he28016675_vent_smok               1
home_labo_clin_clin_ferr                     1
home_dome_do42264188_dome_do10278707         1
Name: count, Length: 1380, dtype: int64

In [54]:
df22_1.shape

(1048565, 16)

In [55]:
df22_1.columns

Index(['Sub Category ID', 'Order Qty', 'Order value',
       'Shipment Quantity Received', 'Shipment Quantity Rejected',
       'Shipment Quantity Accepted', 'CRAC amount', 'Total payments', 'price',
       'Time taken to deliver', 'Purchase Mode_encoded',
       'MSME verfied gender wise_encoded', 'Order MSME Verified_encoded',
       'Startup verfied_encoded', 'Payment Type_encoded',
       'MSME verified social category_encoded'],
      dtype='object')

In [56]:
df33=df22_1[df22_1["Sub Category ID"]=="home_info_comp_comp_de52008682"]

In [58]:
df33.shape

(305420, 16)

In [59]:
df33 = df33.drop(columns=["Sub Category ID"])
I = df33.drop(columns=['Time taken to deliver'])
j = df33['Time taken to deliver']
I_train, I_test, j_train, j_test = train_test_split(I, j, test_size=0.2)
I_train_bil, j_train_bil = np.array(I_train), np.array(j_train)
I_test_bil, j_test_bil = np.array(I_test), np.array(j_test)

print(I_train_bil.shape, j_train_bil.shape, I_test_bil.shape, j_test_bil.shape)

(244336, 14) (244336,) (61084, 14) (61084,)


In [60]:
I_test_bil

array([[2.49200000e+03, 1.09530876e+08, 4.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.49200000e+03, 1.09530876e+08, 4.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.00000000e+03, 2.49810000e+08, 3.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [6.00000000e+03, 2.49810000e+08, 6.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [6.00000000e+03, 2.49810000e+08, 3.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.49200000e+03, 1.09530876e+08, 4.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

## Running LSTM-GRU model on Unseen data

In [61]:
j_pred_gru = model_GRU.predict(I_test_bil)

1909/1909 [==============================] - 5s 2ms/step


In [62]:
mse_gru_unseen = mean_squared_error(j_test, j_pred_gru)
rmse_gru_unseen = np.sqrt(mse_gru_unseen)
print(f"Root Mean Squared Error: {rmse_gru_unseen}")
print(f"Mean Squared Error: {mse_gru_unseen}")

Root Mean Squared Error: 193.67037920764793
Mean Squared Error: 37508.21578243414


In [63]:
# Create a DataFrame of Real and Predicted values
prices_gru_unseen = pd.DataFrame({
    "Real": j_test.ravel(),
    "Predicted": j_pred_gru.ravel()
}, index = df1.index[-len(j_test): ]) 
#prices_gru_unseen.head()
prices_gru_unseen.tail()

,Real,Predicted
1048560,238,33.380970
1048561,238,33.367104
1048562,238,32.546757
1048563,238,33.380970
1048564,214,30.031805


## Catboost Implementation on unseen data

In [64]:
j_pred_cb = model_cb.predict(I_test_bil)

In [65]:
from sklearn.metrics import mean_squared_error
mse_cat_uns = mean_squared_error(j_test_bil, j_pred_cb)
rmse_cat_uns = np.sqrt(mse_cat_uns)
print(f"Root Mean Squared Error: {rmse_cat_uns}")
print(f"Mean Squared Error: {mse_cat_uns}")

Root Mean Squared Error: 195.06662409593508
Mean Squared Error: 38050.987836184846


In [66]:
# Create a DataFrame of Real and Predicted values
prices_catb_unseen = pd.DataFrame({
    "Real": j_test.ravel(),
    "Predicted": j_pred_cb.ravel()
}, index = df33.index[-len(j_test): ]) 
prices_catb_unseen.head()

,Real,Predicted
801745,214,29.660569
801746,214,29.660569
801747,238,29.848272
801748,238,30.172573
801749,214,29.660569
